## Краткое описание работы
Я подумал - наверное же можно объединить все эти 3 упраженения в одно - написать свою реализацию U-Net, которую обучить на другом датасете - а именно на Carvana. <br>
Воодушевлённый такой идеей я начал писать, в каждой проблемой всё больше и больше погружаясь в дебри гугла, в итоге я дошёл до научных статей, где был представлен U-Net (правда почти сразу после этого я нашёл статью, в которой описывается по шагам реализация U-Net (ссылка ниже), поэтому исходные статьи я не читал), и дошёл до репозитория участника этого соревнования, сделал свою реализацию и поставил обучаться. <br>
Сначала я получал ошибки, что на GPU не хватает памяти, затем, позакрывав всё лишнее и изменив параметры обучения (точнее - уменьшил Batch Size, чтобы за раз меньше картинок грузить) я запустил снова, и компьютер начал долго думать. <br>
Через где-то 20-30 минут температура от компа уже несло жаром, что мне конкретно напугало, но программа просмотра температуры говорила, что на видеокарте всего 70 градусов. Надо отметить, что видеокарта у меня 1060 Gaming с 6 Гб видеопамяти, так что, видимо, моя реализация оочень плохая в плане оптимизации. <br>
В итоге я прогнал одну эпоху и получил accuracy на валидационной выборке 0.8183, что, мне кажется, неплохо. Посмотрев датасеты учасников соревнования я увидел, что могут и по 100 эпох обучать.<br>
Кстати, можно будет в итоговом проекте с текстом работать? Мне это ближе, я решал задачу генерации сонетов, как у Шекспира, вот я чем-то подобным хотел бы заняться.

## Задание
<ol>
    <li>Попробуйте обучить нейронную сеть U-Net на любом другом датасете. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?
    </li>
    <li>*Попробуйте свои силы в задаче Carvana на Kaggle - https://www.kaggle.com/c/carvana-image-masking-challenge/overview</li>
    <li>*Сделайте свою реализацию U-Net на TensorFlow</li>
</ol>

Я заручился поддержкой интернета, и нашёл статью с хорошим (на мой взгляд) описанием реализации UNet. <br>
https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5  <br>
Я так понимаю, это каноничная реализация архитектуры (просто по картинке), поэтому особо тут ничего не сделаешь, чтобы это не выглядело, как будто я списал. Однако мне не понравилось название переменных, я их ихменил. Слоя переименовал в соответствии с картинкой - у нас есть как бы 5 слоёв сверху вниз, в каждом из которых есть input и output части, содержащие по несколько свёрточных слоёв, переход между слоями происходит с помощью пулинга и обратной свёрткой. Вот эту структуру я постарался лучше отразить в названиях переменных. Как и в оригинальной работе, макс пулинг я отнёс к предыдущему слою, а обратную свёртку - к следующему слою.<br>
Так же я поменял паддинг, т.к. на картинке размерность уменьшается.<br>
Не понял последний слой - во всех примерах, что я нашёл, используется свёртка с одним фильтром, но на картинке стоит обозначение, как будто используется 2. Полагаю, что это обозначние, что на выходе мы имеем 2 класса, и выходная матрица одна и содержит 1 и 0 принадлежит пиксель объекту или нет.

Конвертация изображений т.к. opencv не работает с gif.

In [40]:
from PIL import Image
import glob

In [43]:
if False:
    for file in glob.glob("carvana-image-masking-challenge/train_masks/*.gif"):
        img = Image.open(file)
        img.save(file[:-3]+".png",'png', optimize=True, quality=95) # В названии ошибся, получилось типа 12345_01..png

In [15]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Cropping2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Построение каноничной U-Net

In [16]:
def build_model(start_filters=64, debug=False):
    input_layer = Input(shape=(572, 572, 3))
    
    l1_conv1 = Conv2D(start_filters*1,(3,3),activation="relu", padding="valid")(input_layer)
    l1_conv2 = Conv2D(start_filters*1,(3,3),activation="relu", padding="valid")(l1_conv1)
    l1_pool = MaxPooling2D((2, 2))(l1_conv2)

    l2_conv1 = Conv2D(start_filters*2,(3,3),activation="relu", padding="valid")(l1_pool)
    l2_conv2 = Conv2D(start_filters*2,(3,3),activation="relu", padding="valid")(l2_conv1)
    l2_pool = MaxPooling2D((2, 2))(l2_conv2)

    l3_conv1 = Conv2D(start_filters*4,(3,3),activation="relu", padding="valid")(l2_pool)
    l3_conv2 = Conv2D(start_filters*4,(3,3),activation="relu", padding="valid")(l3_conv1)
    l3_pool = MaxPooling2D((2, 2))(l3_conv2)

    l4_conv1 = Conv2D(start_filters*8,(3,3),activation="relu", padding="valid")(l3_pool)
    l4_conv2 = Conv2D(start_filters*8,(3,3),activation="relu", padding="valid")(l4_conv1)
    l4_pool = MaxPooling2D((2, 2))(l4_conv2)

    l5_conv1 = Conv2D(start_filters*16,(3,3),activation="relu", padding="valid")(l4_pool)
    l5_conv2 = Conv2D(start_filters*16,(3,3),activation="relu", padding="valid")(l5_conv1)

    l4_deconv = Conv2DTranspose(start_filters * 8, (3, 3), strides=(2, 2), padding="same")(l5_conv2)
    l4_conv2_crop = Cropping2D(cropping=(4, 4))(l4_conv2)
   
    l4_concat = concatenate([l4_deconv, l4_conv2_crop])
    l4_convu1 = Conv2D(start_filters*8,(3,3),activation="relu", padding="valid")(l4_concat)
    l4_convu2 = Conv2D(start_filters*8,(3,3),activation="relu", padding="valid")(l4_convu1)

    l3_deconv = Conv2DTranspose(start_filters*4,(3,3),strides=(2, 2), padding="same")(l4_convu2)
    l3_conv2_crop = Cropping2D(cropping=(16, 16))(l3_conv2)
   
    l3_concat = concatenate([l3_deconv, l3_conv2_crop])
    l3_convu1 = Conv2D(start_filters*4,(3,3),activation="relu", padding="valid")(l3_concat)
    l3_convu2 = Conv2D(start_filters*4,(3,3),activation="relu", padding="valid")(l3_convu1)

    l2_deconv = Conv2DTranspose(start_filters*2,(3,3),strides=(2, 2), padding="same")(l3_convu2)
    l2_conv2_crop = Cropping2D(cropping=(40, 40))(l2_conv2)
   
    l2_concat = concatenate([l2_deconv, l2_conv2_crop])
    l2_convu1 = Conv2D(start_filters*2,(3,3),activation="relu", padding="valid")(l2_concat)
    l2_convu2 = Conv2D(start_filters*2,(3,3),activation="relu", padding="valid")(l2_convu1)

    l1_deconv = Conv2DTranspose(start_filters*1,(3,3),strides=(2, 2), padding="same")(l2_convu2)
    l1_conv2_crop = Cropping2D(cropping=(88, 88))(l1_conv2)
   
    l1_concat = concatenate([l1_deconv, l1_conv2_crop])
    l1_convu1 = Conv2D(start_filters*1,(3,3),activation="relu", padding="valid")(l1_concat)
    l1_convu2 = Conv2D(start_filters*1,(3,3),activation="relu", padding="valid")(l1_convu1)

    output_layer = Conv2D(1,(1,1), padding="same", activation="sigmoid")(l1_convu2)
    
    if debug:    
        print(f"Shape of l1_conv1 = {l1_conv1.shape}")
        print(f"Shape of l1_conv2 = {l1_conv2.shape}")
        print(f"Shape of l1_pool = {l1_pool.shape}")
        print(f"Shape of l2_conv1 = {l2_conv1.shape}")
        print(f"Shape of l2_conv2 = {l2_conv2.shape}")
        print(f"Shape of l2_pool = {l2_pool.shape}")
        print(f"Shape of l3_conv1 = {l3_conv1.shape}")
        print(f"Shape of l3_conv2 = {l3_conv2.shape}")
        print(f"Shape of l3_pool = {l3_pool.shape}")
        print(f"Shape of l4_conv1 = {l4_conv1.shape}")
        print(f"Shape of l4_conv2 = {l4_conv2.shape}")
        print(f"Shape of l4_pool = {l4_pool.shape}")
        print(f"Shape of l5_conv1 = {l5_conv1.shape}")
        print(f"Shape of l5_conv2 = {l5_conv2.shape}")
        print(f"Shape of l4_deconv = {l4_deconv.shape}")
        print(f"Shape of l4_conv2 = {l4_conv2_crop.shape}")
        print(f"Shape of l4_concat = {l4_concat.shape}")
        print(f"Shape of l4_convu1 = {l4_convu1.shape}")
        print(f"Shape of l4_convu2 = {l4_convu2.shape}")
        print(f"Shape of l3_deconv = {l3_deconv.shape}")
        print(f"Shape of l3_conv2 = {l3_conv2_crop.shape}")
        print(f"Shape of l3_concat = {l3_concat.shape}")
        print(f"Shape of l3_convu1 = {l3_convu1.shape}")
        print(f"Shape of l3_convu2 = {l3_convu2.shape}")
        print(f"Shape of l2_deconv = {l2_deconv.shape}")
        print(f"Shape of l2_conv2 = {l2_conv2_crop.shape}")
        print(f"Shape of l2_concat = {l2_concat.shape}")
        print(f"Shape of l2_convu1 = {l2_convu1.shape}")
        print(f"Shape of l2_convu2 = {l2_convu2.shape}")
        print(f"Shape of l1_deconv = {l1_deconv.shape}")
        print(f"Shape of l1_conv2 = {l1_conv2_crop.shape}")
        print(f"Shape of l1_concat = {l1_concat.shape}")
        print(f"Shape of l1_convu1 = {l1_convu1.shape}")
        print(f"Shape of l1_convu2 = {l1_convu2.shape}")
        print(f"Shape of output_layer = {output_layer.shape}")
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [17]:
model = build_model(debug=True)

Shape of l1_conv1 = (None, 570, 570, 64)
Shape of l1_conv2 = (None, 568, 568, 64)
Shape of l1_pool = (None, 284, 284, 64)
Shape of l2_conv1 = (None, 282, 282, 128)
Shape of l2_conv2 = (None, 280, 280, 128)
Shape of l2_pool = (None, 140, 140, 128)
Shape of l3_conv1 = (None, 138, 138, 256)
Shape of l3_conv2 = (None, 136, 136, 256)
Shape of l3_pool = (None, 68, 68, 256)
Shape of l4_conv1 = (None, 66, 66, 512)
Shape of l4_conv2 = (None, 64, 64, 512)
Shape of l4_pool = (None, 32, 32, 512)
Shape of l5_conv1 = (None, 30, 30, 1024)
Shape of l5_conv2 = (None, 28, 28, 1024)
Shape of l4_deconv = (None, 56, 56, 512)
Shape of l4_conv2 = (None, 56, 56, 512)
Shape of l4_concat = (None, 56, 56, 1024)
Shape of l4_convu1 = (None, 54, 54, 512)
Shape of l4_convu2 = (None, 52, 52, 512)
Shape of l3_deconv = (None, 104, 104, 256)
Shape of l3_conv2 = (None, 104, 104, 256)
Shape of l3_concat = (None, 104, 104, 512)
Shape of l3_convu1 = (None, 102, 102, 256)
Shape of l3_convu2 = (None, 100, 100, 256)
Shape of l

In [18]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split

input_size = 572
output_size = 388

epochs = 1
batch_size = 2

orig_width = 1918
orig_height = 1280

threshold = 0.5

df_train = pd.read_csv('carvana-image-masking-challenge/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

print(f'Training on {len(ids_train_split)} samples')
print(f'Validating on {len(ids_valid_split)} samples')

Training on 4070 samples
Validating on 1018 samples


In [19]:
def train_generator():
    while True:
        for start in range(0, len(ids_train_split), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_train_split))
            ids_train_batch = ids_train_split[start:end]
            for id in ids_train_batch.values:
                img = cv2.imread(f'carvana-image-masking-challenge/train/{id}.jpg')
                img = cv2.resize(img, (input_size, input_size))
                mask = cv2.imread(f'carvana-image-masking-challenge/train_masks/{id}_mask..png', cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (output_size, output_size))
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch


def valid_generator():
    while True:
        for start in range(0, len(ids_valid_split), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch.values:
                img = cv2.imread(f'carvana-image-masking-challenge/train/{id}.jpg')
                img = cv2.resize(img, (input_size, input_size))
                mask = cv2.imread(f'carvana-image-masking-challenge/train_masks/{id}_mask..png', cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (output_size, output_size))
                mask = np.expand_dims(mask, axis=2)
                x_batch.append(img)
                y_batch.append(mask)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [20]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
             TensorBoard(log_dir='logs')]

W0427 14:08:29.881245  7856 callbacks.py:1892] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


In [21]:
model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

W0427 14:08:32.236544  7856 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']
W0427 14:08:32.360060  7856 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 2035.0 steps, validate for 509.0 steps
2035/2035 - 1119s - loss: 0.4445 - accuracy: 0.8037 - val_loss: 0.3887 - val_accuracy: 0.8183
